# Preprocessing

Preprocessing to handle all data from the main survey (not prescreen):
- survey data
- Amazon data

.. from both the major survey (v0) & v-cloudresearch surveys...

#### Background

- v0: survey v0 was initially used in mturk and then moved to Prolific. Prolific was the platform used for the large majority of responses
    - survey responses collected from
- v-cloudresearch: 
    - The v0 survey was copied to v-cloudresearch, used on the cloudresearch platform. 
    - The survey questions are the same as v0. 
    - The only difference is the mechanics of collecting worker IDs and routing them back to the platform. 
    - This survey was only used at the end of data collection
    - There are 2 cloudresearch platform products used: mturk toolkit vs connect. This results in worker IDs that look different. The embedded data field "connect" indicates if the worker came from connect.
- 2 surveys?
    - The prolific sample pool was not large enough so we copied the survey to another version used in cloudresearch
    - Note some workers work on both Prolific and Cloudresearch -- making data deduplication extra necessary

## Data inputs and outputs

### Output of preprocessing


#### Survey data
survey-data.csv

Cleaned version of survey data
- Publicly available
- Cleaned to only include relevant fields
- Removed worker IDs (privacy/COUHES compliance)
- Removed comments (privacy/COUHES compliance)
- Filtered to:
    - only participants who passed attention check
    - duplicate responses removed

#### Amazon data
amazon-data.csv

Cleaned version of Amazon data
- one file, where Survey ResponseId field can be used to join with survey data


### Input data

Survey data: 
- Downloaded from Qualtrics filtered to only include successful responses: completed & __passed the attention check__
- Excluding fields:
    - Qualtrics metadata (end time, lat,lon, IP address, etc)
    - Error case related fields -- not relevant for data analysis or participants who successfully completed
    - Embedded data used for survey logic (hidden file field, API token, etc)
    - Embedded data fields used for platform integration

- [Link to input data before preprocessing](https://docs.google.com/spreadsheets/d/13s_z089Im7m22uxC7dVSG5EYOymiIyvsE3RXeWal8ck/edit#gid=914929233) (requires access/share request)

Amazon data:
- All data downloaded from Qualtrics


### Preprocessing steps

1. Clean + Deduplicate Amazon data → Intermediary outputs: bad ResponseIds; cleaner Amazon data
- Make a merged and clean data frame
    - handle prices: 
        - drop any rows with null or non-positive price or quantity
        - prices mapped to floats
    - handle states:
        - state names mapped to consistent set (data contains state names both spelled out and abbreviated, with differing casing)
        - drop data rows for items shipped outside US
    - handle dates:
        - dates mapped to consistent format
        - drop rows for dates before Jan 2018

- Deduplicate: 
    - use initial rows of data from jan 2018
    - Find # of dupes
    - Drop duplicates. How to choose which to drop vs keep? Keep version with the most rows that have no nan values (they completed survey later or data processed/downloaded with more success).
    - Collect response ids for duplicates to drop from both Amazon and survey data → output: drop ResponseIds

2. Process survey data → Output: processed survey data (can be used on its own)
- Deduplicate on worker/Prolific IDs
- Join the 2 survey datasets
- Remove data where 'test' is indicated
- Remove rows with bad ResponseIds (from step 1)
- Make public
    - separate comments
    - remove worker/Prolific IDs

3. Finish Amazon data preprocessing → Output: cleaned Amazon data
- Exclude rows without a ResponseId in survey data
- Keep for internal use for now


### Notes from the process

- There are data for (66) response ids where there is no valid US state associated with any of their purchases. These response ids are kept. 
    - Note the possible reason for no state: shipped outside the US, nan value because delivered to locker or digital good (e.g. gift card, software)
    
- During process of deduping found that sometimes items were recategorized. E.g. early version of data had category = HOME; later version THERMOMETER.


In [31]:
from datetime import date, datetime
import os

from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [207]:
# Collect the bad Response Ids to remove from data
drop_responseids = []

### Process Amazon data

1. Clean + Deduplicate Amazon data → Intermediary outputs: drop ResponseIds; cleaner Amazon data
- Make a merged and clean data frame

- Deduplicate: 
    - use initial rows of data from jan 2018
    - Record ResponseIds for dupes to remove from survey data → output: bad ResponseIds

In [6]:
# Get the filepaths

amzn_input_v0_fpath = r'../data/amazon-data/unprocessed/v0/'
amzn_input_vcloudresearch_fpath = r'../data/amazon-data/unprocessed/v-cloudresearch/'
amzn_input_v0_fnames = [amzn_input_v0_fpath+f for f in os.listdir(amzn_input_v0_fpath) if f.endswith('.csv')]
amzn_input_vcloudresearch_fnames = [amzn_input_vcloudresearch_fpath+f for f in os.listdir(amzn_input_vcloudresearch_fpath) if f.endswith('.csv')]
print('%s v0 files' % len(amzn_input_v0_fnames))
print('%s v-cloudresearch files' % len(amzn_input_vcloudresearch_fnames))
amzn_input_fnames = amzn_input_v0_fnames + amzn_input_vcloudresearch_fnames
print('%s total files to read' % len(amzn_input_fnames))
amzn_input_fnames[:3]

4645 v0 files
612 v-cloudresearch files
5257 total files to read


['../data/amazon-data/unprocessed/v0/R_hrzkhELGx5jezT7_R_3I9Pu8iauEcOx9A.csv',
 '../data/amazon-data/unprocessed/v0/R_c2JhEelp5x2LnDO_R_7UtikIBqeQHvnyN.csv',
 '../data/amazon-data/unprocessed/v0/R_9kK8zHlub96lJvk_R_2PpVNZupgA2k9mX.csv']

In [7]:
def get_unit_price_float(unit_price):
    try:
        return float(unit_price.strip('$').replace(',',''))
    except Exception as e:
        #print('Exception:', e, 'for:', unit_price)
        return 0

In [60]:
# The Order Date has multiple formats
# Weird ones found: 03-Jun-18, 2001/4/18
def parse_date(text):
    """Parses date or returns nan"""
    text = str(text)
    for fmt in ('%m/%d/%y', '%m/%d/%Y', '%d-%b-%y'):
        try:
            return datetime.strptime(text, fmt)
        except ValueError:
            pass
    print('no valid date format found for %s' % text)
    return np.nan

In [64]:
"""
Make a giant data frame:
- map date to consistent format
- drop any rows with null or non-positive price or quantity
- drop rows before 2018-01-01
- prices mapped to floats
- dates mapped to yyyy-mm
"""

files_used = 0
amzn_df = pd.DataFrame()

for i, f in enumerate(amzn_input_fnames):
    if i%500 == 0:
        print('%s: reading file %s - %s files used; %s total purchases' % 
              (pd.Timestamp.now(), i, files_used, len(amzn_df)))
    df = pd.read_csv(f)
    files_used += 1
    # Handle dates
    df['Order Date'] = df['Order Date'].apply(parse_date)
    if len(df[df['Order Date'].isna()]) > 0:
        print('Found %s rows with null Order Date' % len(df[df['Order Date'].isna()]))
        df = df[df['Order Date'].notna()]   
    # Drop rows before 1-1-2018 
    df = df[df['Order Date'] >= pd.Timestamp('2018-01-01')]
    # Drop rows where there is null or non-positive price or quantity
    # First handle weird values (found Quantities with '$')
    df['Quantity'] = df['Quantity'].astype(str).apply(lambda s: int(s) if s.isnumeric() else np.nan)
    df = df[(df.notna()['Purchase Price Per Unit'] | df.notna()['Quantity'])]
    # extract unit price and total price as floats
    df['unit price'] = df['Purchase Price Per Unit'].apply(get_unit_price_float)
    df['total price'] = df['unit price'] * df['Quantity']
    df = df[df['total price'] > 0]
    df['yyyy-mm'] = df['Order Date'].apply(lambda d: date.strftime(d, '%Y-%m'))
    amzn_df = pd.concat([amzn_df, df])

2023-04-20 11:29:49.138948: reading file 0 - 0 files used; 0 total purchases
2023-04-20 11:29:53.018147: reading file 500 - 500 files used; 174675 total purchases
2023-04-20 11:30:01.617452: reading file 1000 - 1000 files used; 379981 total purchases
2023-04-20 11:30:15.309829: reading file 1500 - 1500 files used; 564092 total purchases
2023-04-20 11:30:32.709280: reading file 2000 - 2000 files used; 753235 total purchases
2023-04-20 11:30:54.594418: reading file 2500 - 2500 files used; 935331 total purchases
2023-04-20 11:31:20.846200: reading file 3000 - 3000 files used; 1116055 total purchases
2023-04-20 11:31:51.503982: reading file 3500 - 3500 files used; 1288570 total purchases
2023-04-20 11:32:27.547795: reading file 4000 - 4000 files used; 1472125 total purchases
no valid date format found for nan
Found 1 rows with null Order Date
2023-04-20 11:33:07.707788: reading file 4500 - 4500 files used; 1645908 total purchases
no valid date format found for 2001/4/18
no valid date forma

In [68]:
def print_data_metrics(amazon_data_df):
    print('%s response Ids' % amazon_data_df['Survey ResponseID'].nunique())
    print('%s total rows of data' % len(amazon_data_df))

In [70]:
print_data_metrics(amzn_df)
# amzn_df.head()

5254 response Ids
1942117 total rows of data


### Handle US states

Map U.S. state names to consistent set of 52, or nan
- Nan can happen when gift cards are purchased
- Nan can happen when items are delivered to a locker

Drop purchases not made in the US: Drop if state is neither nan nor in set of US states

In [112]:
import re

states_df = pd.read_csv('../data/census/state-abbreviations.csv')
states_df['index'] = states_df['state'].apply(lambda s: s.upper())
states_df = states_df.set_index('index')
state_codes = states_df['code'].to_list()
state_names_map = states_df['code'].to_dict()

print('states df contains %s states'%len(states_df))

def state_code(s):
    if not isinstance(s, str): # probably nan (float)
        return np.nan
    s = re.sub(r'\.', '', s).upper()
    if s in state_codes:
        return s
    if s in state_names_map:
        return state_names_map[s]
    return np.nan

states_df.head(3)

states df contains 52 states


,state,abbrev,code
index,,,
ALABAMA,Alabama,Ala.,AL
ALASKA,Alaska,Alaska,AK
ARIZONA,Arizona,Ariz.,AZ


In [113]:
# use a consisent set of state names
amzn_df['state'] = amzn_df['Shipping Address State'].apply(state_code)

In [101]:
snames = amzn_df['Shipping Address State'].unique()
print('%s shippping address states'%len(snames))
snames

910 shippping address states


array(['RHODE ISLAND', nan, 'SC', 'AL', 'TN', 'MO', 'NY', 'New York',
       'TX', 'IL', 'CO', 'CA', 'Mo', 'WI', 'KS', 'PA', 'Pennsylvania',
       'MI', 'IA', 'CT', 'Colorado', 'VA', 'FL', 'OH', 'GA', 'WA', 'DE',
       'VT', 'MN', 'AZ', 'Az', 'Arizona', 'VIC', 'Texas', 'LOUISIANA',
       'LA', 'NC', 'DC', 'ID', 'MT', 'Nevada', 'NV', 'South Carolina',
       'NJ', 'MA', 'Ia', 'TEXAS', 'OR', 'WISCONSIN', 'OK', 'PENNSYLVANIA',
       'North Carolina', 'ND', 'ILLINOIS', 'ca', 'California', 'KY', 'IN',
       'NM', 'New Jersey', 'RI', 'PR', 'MS', 'WV', 'NH', 'AR', 'MD', 'ME',
       'SD', 'NE', 'UT', 'Illinois', 'Maryland', 'Massachusetts',
       'Washington', 'Ca', 'Wisconsin', 'New york', 'VERMONT',
       'CALIFORNIA', 'Ma', 'Mississippi', 'Oklahoma', 'FLORIDA', 'HI',
       'PUEBLA', 'Connecticut', 'TENNESSEE', 'Oregon', 'Ontario', 'Ohio',
       'New Brunswick', 'AK', 'ARIZONA', 'North carolina', 'Florida',
       'COAHUILA DE ZARAGOZA', 'OREGON', 'Va', 'Tennessee', 'Alabama',
    

In [116]:
no_state_df = amzn_df[amzn_df['state'].isna()].copy()
print('%s purchases without a US state' % len(no_state_df))
no_state_df['Shipping Address State'] = no_state_df['Shipping Address State'].astype(str)
print(no_state_df.groupby(
    'Shipping Address State'
).count()['Order Date'].sort_values(ascending=False))
# no_state_df.head()

98928 purchases without a US state
Shipping Address State
nan                         96684
AE                            297
AP                            248
                              190
AA                            181
                            ...  
AMZN_US(TBA304302031854)        1
AMZN_US(TBA304289237047)        1
AMZN_US(TBA304245182789)        1
AMZN_US(TBA303503759702)        1
oxfordshire                     1
Name: Order Date, Length: 678, dtype: int64


In [121]:
# There are a lot of shipping codes
# I think someone updated the columns of their data and accidentally replaced Shipping Address State with this shipping info
# Only 2 respondents. So we don't handle these.
shipping_codes = ['USPS(9361289678092208744362)', 'USPS(9200192148989923110961)', 'AMZN_US(TBA681177717000)', 'AMZN_US(TBA681525694000)', 'AMZN_US(TBA709834188000)', 'USPS(9405511699000502074243)', 'AMZN_US(TBA744430486000)', 'AMZN_US(TBA743934442000)', 'UPS(1Z9R65E90230305357)', 'AMZN_US(TBA746521516000)', 'USPS(9400111899223591622989)', 'USPS(9374889701090065272069)', 'AMZN_US(TBA775268283000)', 'AMZN_US(TBA805733051000)', 'USPS(9305520111402562591454)', 'AMZN_US(TBA836724144000)', 'FedEx(74893150559678452829)', 'AMZN_US(TBA873071575000)', 'UPSM(9274890207416100940865)', 'ONTRAC(C12340628189808)', 'AMZN_US(TBA876042282000)', 'AMZN_US(TBA903879047000)', 'AMZN_US(TBA906639505000)', 'AMZN_US(TBA920600142000)', 'AMZN_US(TBA920230439000)', 'AMZN_US(TBA924025854000)', 'UPS(1Z3Y13E20225326465)', 'AMZN_US(TBA940976895000)', 'USPS(9400110899533049617621)', 'AMZN_US(TBA968346911000)', 'UPS(1Z306A400285100411)', 'AMZN_US(TBA003546173000)', 'USPS(9374869903502875776234)', 'AMZN_US(TBA010533615000)', 'UPS Mail Innovations(92748963438659543476676155)', 'AMZN_US(TBA037391402000)', 'AMZN_US(TBA037032059000)', 'UPS(1ZE1260F0285095732)', 'UPSMI(92748999983540543537184677)', 'FEDEX(730893626729)', 'USPS(9374889701090126417965)', 'USPS(9400110298370551431546)', 'FEDEX(730713743768)', 'AMZN_US(TBA057888642000)', 'AMZN_US(TBA057863141000)', 'USPS(9400110897877001992213)', 'USPS(9374889701090128717902)', 'AMZN_US(TBA062268622000)', 'AMZN_US(TBA066392536000)', 'AMZN_US(TBA069469989000)', 'AMZL(TBA077884413000)', 'AMZN_US(TBA072424447000)', 'USPS(9200190213413100117669)', 'USPS(9374889701090130183443)', 'AMZN_US(TBA081604656000)', 'UPS(1ZA475Y40228221420)', 'AMZN_US(TBA083799068000)', 'AMZN_US(TBA093140128000)', 'USPS(9400110200864069403157)', 'USPS(00040899563096726435)', 'USPS(9300120111402821752920)', 'AMZN_US(TBA103764818000)', 'AMZN_US(TBA105007362000)', 'AMZN_US(TBA157286122000)', 'USPS(9374889701090137085481)', 'USPS(9400111699000633256560)', 'AMZN_US(TBA137528290000)', 'AMZN_US(TBA140153719000)', 'AMZN_US(TBA148561738000)', 'AMZN_US(TBA183932354000)', 'AMZN_US(TBA184104175000)', 'AMZN_US(TBA191255433000)', 'USPS(9200192148989951470228)', 'USPS(9400111899560154377949)', 'FEDEX(730895469730)', 'USPS(9400111899223148158138)', 'AMZN_US(TBA209402160000)', 'ONTRAC(C12340757923929)', 'AMZN_US(TBA226374238000)', 'AMZN_US(TBA230261988000)', 'AMZN_US(TBA240288946000)', 'AMZN_US(TBA242881284000)', 'AMZN_US(TBA248366913000)', 'AMZN_US(TBA271046373000)', 'AMZN_US(TBA279319497000)', 'USPS(9361289701090171686372)', 'AMZN_US(TBA292586131000)', 'AMZN_US(TBA317722647000)', 'USPS(9305520111403007184736)', 'USPS(9300120111403030405478)', 'USPS(9374889701090181753428)', 'AMZN_US(TBA357653995000)', 'AMZN_US(TBA361101850000)', 'AMZN_US(TBA356643823000)', 'USPS(9274890233818100325102)', 'AMZN_US(TBA369339822000)', 'AMZN_US(TBA371437034000)', 'USPS(9361289701090183906406)', 'AMZN_US(TBA378341362000)', 'AMZN_US(TBA404359914000)', 'AMZN_US(TBA403925267000)', 'AMZN_US(TBA404429868000)', 'AMZN_US(TBA421521647000)', 'AMZN_US(TBA427303101000)', 'AMZN_US(TBA444801646000)', 'USPS(9374889701090194577172)', 'AMZN_US(TBA448421142000)', 'AMZN_US(TBA449093092000)', 'USPS(9300120111403099298233)', 'AMZN_US(TBA455782242000)', 'AMZN_US(TBA459854374000)', 'USPS(9341989701090198258366)', 'AMZN_US(TBA508089224000)', 'AMZN_US(TBA514305924000)', 'UPS(1Z8000V60276308398)', 'AMZN_US(TBA526262514000)', 'AMZN_US(TBA574070617000)', 'AMZN_US(TBA575226724000)', 'AMZN_US(TBA654333583000)', 'AMZN_US(TBA625304428000)', 'AMZN_US(TBA624080505000)', 'AMZN_US(TBA654402946000)', 'USPS(9200190224060823286309)', 'AMZN_US(TBA630068771000)', 'UPS(1Z0915FEPW14379119)', 'USPS(9200190224054036609147)', 'AMZN_US(TBA676729107000)', 'FEDEX(784903992766)', 'USPS(9374889701090239422467)', 'USPS(9200190224058233201722)', 'USPS(9361289701090238857653)', 'AMZN_US(TBA684169456000)', 'AMZN_US(TBA731268445000)', 'AMZN_US(TBA785923953000)', 'AMZN_US(TBA798033270000)', 'AMZN_US(TBA803195767000)', 'USPS(9400111699000838695553)', 'AMZN_US(TBA838540166000)', 'AMZN_US(TBA838959749000)', 'AMZN_US(TBA836153917000)', 'AMZN_US(TBA874702490000)', 'AMZN_US(TBA871351174000)', 'USPS(9374889701090260180008)', 'USPS(9200190224058251447690)', 'AMZN_US(TBA892486616000)', 'AMZN_US(TBA901828211000)', 'AMZN_US(TBA893893560000)', 'AMZN_US(TBA893641960000)', 'AMZN_US(TBA894903259000)', 'AMZN_US(TBA902985910000)', 'AMZN_US(TBA907766064000)', 'AMZN_US(TBA951272479000)', 'AMZN_US(TBA952337711000)', 'UPS(1Z427YA44424304184)', 'AMZN_US(TBA010683229000)', 'Royal Mail', 'AMZN_US(TBA068691029000)', 'AMZN_US(TBA068485939000)', 'AMZN_US(TBA072176294000)', 'AMZN_US(TBA131035902000)', 'SUREPOST(1Z81E22YYN01248448)', 'AMZN_US(TBA131441931000)', 'USPS(9400111699000370326625)', 'UPS Mail Innovations(92748901790731543400809651)', 'AMZN_US(TBA198774978000)', 'AMZN_US(TBA203016098000)', 'USPS(9374889701090283852562)', 'AMZN_US(TBA217194772000)', 'AMZN_US(TBA199323661000)', 'AMZN_US(TBA199244798000)', 'AMZN_US(TBA203138897000)', 'AMZN_US(TBA205747154000)', 'AMZN_US(TBA206588392000)', 'AMZN_US(TBA205492245000)', 'AMZN_US(TBA268198157000)', 'AMZN_US(TBA271066443000)', 'AMZN_US(TBA273616272000)', 'AMZN_US(TBA274543266000)', 'AMZN_US(TBA275261050000)', 'AMZN_US(TBA344157757000)', 'AMZN_US(TBA342423408000)', 'AMZN_US(TBA351967189000)', 'AMZN_US(TBA393923450000)', 'AMZN_US(TBA394255505000)', 'AMZN_US(TBA393998037000)', 'AMZN_US(TBA395931186000)', 'UPS(1Z0929RY0231124905)', 'AMZN_US(TBA415249264000)', 'AMZN_US(TBA412272818000)', 'UPS(1Z552RX80300662713)', 'AMZN_US(TBA410668046000)', 'AMZN_US(TBA413072909000)', 'AMZN_US(TBA410849199000)', 'AMZN_US(TBA411083308000)', 'UPS(1Z2W4E841330035285)', 'AMZN_US(TBA424965148000)', 'UPS(1Z9728Y90310679778)', 'AMZN_US(TBA433811678000)', 'AMZN_US(TBA437095839000)', 'AMZN_US(TBA443035222000)', 'UPSM(9274899993700911385179)', 'AMZN_US(TBA471063914000)', 'AMZN_US(TBA493785711000)', 'AMZN_US(TBA479877978000)', 'AMZN_US(TBA482955426000)', 'UPS(1Z076AW90330378537)', 'AMZN_US(TBA486786260000)', 'UPS(1ZY182671337080144)', 'AMZN_US(TBA486383235000)', 'Japan Post', 'AMZN_US(TBA487129105000)', 'AMZN_US(TBA489379865000)', 'AMZN_US(TBA490249506000)', 'AMZN_US(TBA493136824000)', 'AMZN_US(TBA492334970000)', 'AMZN_US(TBA514600009000)', 'AMZN_US(TBA594992727000)', 'UPS(1ZY182671339615107)', 'UPS(1ZY182670239919612)', 'AMZN_US(TBA619556435000)', 'EUB(LW932225509CN)', 'AMZN_US(TBA620294373000)', 'EUB(LW934014989CN)', 'USPS(9361289701090314398650)', 'AMZN_US(TBA653155334000)', 'AMZN_US(TBA687246718000)', 'AMZN_US(TBA686445717000)', 'AMZN_US(TBA693565842000)', 'AMZN_US(TBA749488214000)', 'USPS(9200192148989972355474)', 'AMZN_US(TBA803571992000)', 'AMZN_US(TBA813802123000)', 'USPS(9200190224058160410679)', 'AMZN_US(TBA814865310000)', 'AMZN_US(TBA830386880000)', 'AMZN_US(TBA844448446000)', 'AMZN_US(TBA859842611000)', 'AMZN_US(TBA860781574000)', 'AMZN_US(TBA874490632000)', 'AMZN_US(TBA877005255000)', 'USPS(9400110298192004710288)', 'USPS(9200190224054082316815)', 'UPS(1Z975W5A0215322136)', 'AMZN_US(TBA891218863000)', 'AMZN_US(TBA891619798000)', 'AMZN_US(TBA899654208000)', 'AMZN_US(TBA912717950000)', 'DHL Global Mail(9374869903504139917988)', 'AMZN_US(TBA947681637000)', 'AMZN_US(TBA948690845000)', 'AMZN_US(TBA948196826000)', 'AMZN_US(TBA947547989000)', 'AMZN_US(TBA953513296000)', 'AMZN_US(TBA958142160000)', 'AMZN_US(TBA963094018000)', 'AMZN_US(TBA959238672000)', 'AMZN_US(TBA971278516000)', 'UPS(1Z5Y68W50329134180)', 'AMZN_US(TBA982613177000)', 'AMZN_US(TBA983140185000)', 'AMZN_US(TBA986948925000)', 'AMZN_US(TBA989463220000)', 'AMZN_US(TBA995333823000)', 'AMZN_US(TBA033009637000)', 'AMZN_US(TBA036035784000)', 'USPS(9200190224058200827399)', 'AMZN_US(TBA043212869000)', 'UPS(1Z5Y68W50329842870)', 'AMZN_US(TBA060673643000)', 'AMZN_US(TBA103615028000)', 'AMZN_US(TBA122053709000)', 'AMZN_US(TBA121120605000)', 'AMZN_US(TBA127892260000)', 'AMZN_US(TBA130592053000)', 'AMZN_US(TBA169756504000)', 'AMZN_US(TBA214187612000)', 'AMZN_US(TBA213497110000)', 'AMZN_US(TBA241814408000)', 'AMZN_US(TBA247505398000)', 'AMZN_US(TBA250037272000)', 'UPS(1Z9YF2121302177359)', 'USPS(9374889701090368352369)', 'AMZN_US(TBA346027766000)', 'USPS(9374889701090372464126)', 'USPS(9374889701090373289759)', 'AMZN_US(TBA332792092000)', 'OSM BPM Mail(9241990226079704034145)', 'AMZN_US(TBA356644977000)', 'AMZN_US(TBA357893652000)', 'UPS(1Z5Y68W50333867210)', 'AMZN_US(TBA448275470000)', 'AMZN_US(TBA481063621000)', 'AMZN_US(TBA489719558000)', 'USPS(9374889701090399493888)', 'AMZN_US(TBA478900117000)', 'AMZN_US(TBA501238150000)', 'AMZN_US(TBA548503484000)', 'USPS(9200190224060792526901)', 'AMZN_US(TBA550408832000)', 'AMZN_US(TBA549587372000)', 'AMZN_US(TBA637760892000)', 'AMZN_US(TBA637807520000)', 'AMZN_US(TBA636401708000)', 'AMZN_US(TBA686241599000)', 'AMZN_US(TBA687877758000)', 'China Post(LS225189489CN)', 'AMZN_US(TBA687957647000)', 'AMZN_US(TBA694946073000)', 'AMZN_US(TBA690035871000)', 'AMZN_US(TBA692610544000)', 'USPS(9374889701090423372455)', 'AMZN_US(TBA689103347000)', 'AMZN_US(TBA688986315000)', 'AMZN_US(TBA693488006000)', 'AMZN_US(TBA703278366000)', 'USPS(9361289701090425029672)', 'AMZN_US(TBA774479172000)', 'USPS(9361289701090429944735)', 'UPS(1Z129F380262772186)', 'AMZN_US(TBA844614847000)', 'USPS(9374889701090436786195)', 'AMZN_US(TBA864790689000)', 'AMZN_US(TBA872260500000)', 'AMZN_US(TBA879154361000)', 'USPS(9361289701090437749308)', 'USPS(9300120111404315488353)', 'AMZN_US(TBA894845375000)', 'AMZN_US(TBA902282837000)', 'AMZN_US(TBA942815697000)', 'USPS(9374889701090459329102)', 'USPS(9374889701090462659753)', 'USPS(9361289701090461760010)', 'AMZN_US(TBA150881952000)', 'USPS(9400111899223332531822)', 'UPS(1Z8Y82X01340994154)', 'USPS(9374889701090464036293)', 'AMZN_US(TBA208656793000)', 'USPS(LS301497651CN)', 'AMZN_US(TBA004001186101)', 'AMZN_US(TBA004700801401)', 'USPS(9400109205568610731537)', 'AMZN_US(TBA006151018801)', 'USPS(9300120111404508084607)', 'USPS(9361289701090479413199)', 'UPS(1Z9YF1801316709097)', 'UPS(1Z6A390F0304198979)', 'USPS(9374889701090483032153)', 'USPS(9374889701090481289023)', 'FedEx(392825145091)', 'USPS(9374889701090488433528)', 'UPS(1Z9X66Y31352337152)', 'AMZN_US(TBA029357560801)', 'USPS(9361289701090493865417)', 'AMZN_US(TBA025274229601)', 'USPS(9400111899281881926605)', 'UPS(1Z9YF1891303497717)', 'UPS(1Z15998R0317190560)', 'AMZN_US(TBA036761141001)', 'USPS(9300120111404792467278)', 'AMZN_US(TBA040494227301)', 'AMZN_US(TBA040425385901)', 'UPS(1Z6A390F0305155610)', 'USPS(9361289701090506803542)', 'AMZN_US(TBA042754405201)', 'AMZN_US(TBA048869243901)', 'AMZN_US(TBA046900754401)', 'AMZN_US(TBA047003598901)', 'AMZN_US(TBA047539470101)', 'AMZN_US(TBA047397177101)', 'AMZN_US(TBA049654807601)', 'USPS(9305520111499570375787)', 'AMZN_US(TBA051188470701)', 'AMZN_US(TBA050967187101)', 'AMZN_US(TBA055193991901)', 'AMZN_US(TBA056700526901)', 'AMZN_US(TBA060878893201)', 'AMZN_US(TBA066703245001)', 'FedEx(394207454518)', 'UPS(1Z9X26970310004807)', 'USPS(9405510298370122363651)', 'AMZN_US(TBA072183268801)', 'AMZN_US(TBA071691520801)', 'AMZN_US(TBA072093653701)', 'USPS(9374889701090535310215)', 'AMZN_US(TBA077775691801)', 'FedEx(903208534149)', 'UPS(1Z6A390F0306442165)', 'AMZN_US(TBA079406043001)', 'AMZN_US(TBA079733507601)', 'AMZN_US(TBA080599805701)', 'AMZN_US(TBA083720446801)', 'USPS(9374889701090542394895)', 'AMZN_US(TBA084098201501)', 'AMZN_US(TBA083702459301)', 'AMZN_US(TBA086465706101)', 'AMZN_US(TBA086543390401)', 'AMZN_US(TBA086660823801)', 'UPS(1Z1480AE0380261162)', 'AMZN_US(TBA085293368301)', 'UPS(1Z1480AE0380317647)', 'AMZN_US(TBA084772312001)', 'AMZN_US(TBA085836370001)', 'AMZN_US(TBA087005072201)', 'USPS(9374889701090548052102)', 'AMZN_US(TBA089072160501)', 'AMZN_US(TBA089153895101)', 'AMZN_US(TBA089669511601)', 'AMZN_US(TBA089650464701)', 'AMZN_US(TBA089782605401)', 'AMZN_US(TBA090557303201)', 'USPS(9400111899563207215533)', 'AMZN_US(TBA090816293801)', 'AMZN_US(TBA091013887701)', 'AMZN_US(TBA092548962901)', 'AMZN_US(TBA100090152001)', 'AMZN_US(TBA105068839101)', '4PX(302781137668)', 'USPS(9400110298192008940919)', 'AMZN_US(TBA111091323401)', 'AMZN_US(TBA113765470601)', 'AMZN_US(TBA116003373701)', 'USPS(9374889701090584846741)', 'AMZN_US(TBA125223053701)', 'AMZN_US(TBA126398841501)', 'UPS(1Z6A390F0307838236)', 'AMZN_US(TBA129388416401)', 'USPS(9374889701090589027435)', 'USPS(9374889701090589226876)', 'USPS(9374889701090589246003)', 'AMZN_US(TBA132155339101)', 'USPS(9374889701090592769360)', 'USPS(9374889701090592349807)', 'USPS(9374889701090591642763)', 'USPS(9400110298192009246416)', 'USPS(9374889701090591799085)', 'UPS(1Z69VR561372467178)', 'USPS(9374889701090594651823)', 'UPS(1Z593Y0R0312416163)', 'AMZN_US(TBA145688704801)', 'AMZN_US(TBA145540727101)', 'USPS(9374889701090605090351)', 'AMZN_US(TBA000692045204)', 'AMZN_US(TBA147526077501)', 'AMZN_US(TBA000524310004)', 'AMZN_US(TBA003132328104)', 'USPS(9361289701090609529103)', 'FedEx(396946093833)', 'AMZN_US(TBA015010230904)', 'AMZN_US(TBA015581812604)', 'AMZN_US(TBA017345009204)', 'AMZN_US(TBA018351366504)', 'AMZN_US(TBA028638960104)', 'USPS(9405511202555702823996)', 'AMZN_US(TBA029468238304)', 'AMZN_US(TBA029696500904)', 'AMZN_US(TBA030751639804)', 'AMZN_US(TBA035539068004)', 'AMZN_US(TBA040471330604)', 'DHL Global Mail(420890159361269903505714542940)', 'AMZN_US(TBA038587859704)', 'AMZN_US(TBA041906295104)', 'AMZN_US(TBA043377022004)', 'AMZN_US(TBA044160327404)', 'AMZN_US(TBA044450021504)', 'USPS(9374889701090646225910)', 'AMZN_US(TBA066754205504)', 'AMZN_US(TBA067339747304)', 'AMZN_US(TBA066029544504)', 'AMZN_US(TBA070142657204)', 'AMZN_US(TBA069548946904)', 'AMZN_US(TBA070110653004)', 'AMZN_US(TBA071053531404)', 'AMZN_US(TBA070658997704)', 'Amazon(TBA071244976404)', 'UPS(1Z81WV730209855584)', 'AMZN_US(TBA071017664404)', 'AMZN_US(TBA072113924404)', 'USPS(9374889701090654584146)', 'AMZN_US(TBA092544601804)', 'AMZN_US(TBA093376155404)', 'AMZN_US(TBA093902303604)', 'AMZN_US(TBA094429025504)', 'AMZN_US(TBA113130056904)', 'AMZN_US(TBA116696148004)', 'USPS(9374889701090665921954)', 'AMZN_US(TBA119502153104)', 'UPS(1Z6A390F0310967573)', 'AMZN_US(TBA293359968000)', 'AMZN_US(TBA133884045604)', 'AMZN_US(TBA135218514604)', 'AMZN_US(TBA138334877704)', 'AMZN_US(TBA143484611204)', 'USPS(9374889701090675476208)', 'AMZN_US(TBA143177920404)', 'AMZN_US(TBA148077460904)', 'AMZN_US(TBA148849284004)', 'AMZN_US(TBA151651824204)', 'AMZN_US(TBA152068507004)', 'AMZN_US(TBA159036852304)', 'AMZN_US(TBA162569476304)', 'AMZN_US(TBA162698817204)', 'AMZN_US(TBA163221560904)', 'USPS(9374889701090682897980)', 'AMZN_US(TBA167442891204)', 'AMZN_US(TBA167240528904)', 'AMZN_US(TBA169378095004)', 'AMZN_US(TBA169252733804)', 'AMZN_US(TBA173620131004)', 'AMZN_US(TBA173843026204)', 'AMZN_US(TBA174771282404)', 'AMZN_US(TBA174695161104)', 'AMZN_US(TBA175364827004)', 'AMZN_US(TBA177943890704)', 'AMZN_US(TBA175751600104)', 'AMZN_US(TBA186135853704)', 'AMZN_US(TBA307903734000)', 'AMZN_US(TBA190040706504)', 'AMZN_US(TBA002918623804)', 'AMZN_US(TBA002387195304)', 'AMZN_US(TBA001571427404)', 'AMZN_US(TBA008564544104)', 'AMZN_US(TBA007051891204)', 'AMZN_US(TBA007099140204)', 'UPS(1Z6A390F0312665538)', 'AMZN_US(TBA016352822204)', 'AMZN_US(TBA317059549000)', 'DHL(420890159374869903506579219226)', 'UPS(1Z813R301315802704)', 'AMZN_US(TBA022380592304)', 'USPS(9405511108296502696476)', 'AMZN_US(TBA022059533404)', 'AMZN_US(TBA024483246504)', 'AMZN_US(TBA024758015804)', 'AMZN_US(TBA024583717004)', 'AMZN_US(TBA033854603104)', 'AMZN_US(TBA027400037204)', 'AMZN_US(TBA028597269104)', 'AMZN_US(TBA320364305000)', 'AMZN_US(TBA029276675404)', 'AMZN_US(TBA073275445404)', 'AMZN_US(TBA074528516004)', 'AMZN_US(TBA104805001204)', 'AMZN_US(TBA104938336804)', 'AMZN_US(TBA104969131104)', 'AMZN_US(TBA106180684204)', 'AMZN_US(TBA108139064004)', 'AMZN_US(TBA364242644000)', 'AMZN_US(TBA164889458504)', 'AMZN_US(TBA409211406000)', 'AMZN_US(TBA043054368404)', 'AMZN_US(TBA043527956804)', 'AMZN_US(TBA044328668404)', 'AMZN_US(TBA044438320304)', 'USPS(9374889701006743483641)', 'AMZN_US(TBA060629386304)', 'AMZN_US(TBA063020768904)', 'AMZN_US(TBA078948672604)', 'USPS(9374889701007777117380)', 'AMZN_US(TBA103348320004)', 'AMZN_US(TBA105350579504)', 'AMZN_US(TBA110573918804)', 'AMZN_US(TBA144061479304)', 'AMZN_US(TBA146091380304)', 'USPS(9374889701009200082036)', 'AMZN_US(TBA179779212004)', 'AMZN_US(TBA182502864104)', 'AMZN_US(TBA199220151504)', 'AMZN_US(TBA029565427704)', 'AMZN_US(TBA050665026904)', 'USPS(9374889701010608089831)', 'AMZN_US(TBA127094279804)', 'AMZN_US(TBA129611905004)', 'AMZN_US(TBA133380963204)', 'AMZN_US(TBA133446596504)', 'AMZN_US(TBA039295092104)', 'AMZN_US(TBA039767447804)', 'AMZN_US(TBA055755756304)', 'AMZN_US(TBA049317790304)', 'UPS(1Z82V7481340298048)', 'AMZN_US(TBA303310676561)', 'UPS(1Z9A403R0325831732)', 'AMZN_US(TBA303503759702)', 'AMZN_US(TBA777856753000)', 'AMZN_US(TBA779076924000)', 'AMZN_US(TBA303669955025)', 'UPS(1Z83797X0285719924)', 'AMZN_US(TBA304195520918)', 'AMZN_US(TBA304245182789)', 'AMZN_US(TBA304388656735)', 'AMZN_US(TBA304289237047)', 'AMZN_US(TBA304302031854)', 'AMZN_US(TBA304343019656)', 'AMZN_US(TBA304328384372)', 'UPS(1Z4E3W290315382658)', 'AMZN_US(TBA304641260603)', 'AMZN_US(TBA304730455450)', 'AMZN_US(TBA304713399048)', 'AMZN_US(TBA304723766586)', 'AMZN_US(TBA304834242476)', 'AMZN_US(TBA304889394828)', 'AMZN_US(TBA304966534152)', 'AMZN_US(TBA304960015006)', 'Yanwen(UG878348002YP)', 'AMZN_US(TBA305009967564)', 'AMZN_US(TBA305021479645)', 'USPS(9374889720118171897114)', 'AMZN_US(TBA305388485956)', 'SF Express(SF6043545498225)']
print('%s shipping codes' % len(shipping_codes))
print('%s unique Response Ids with shipping codes' % no_state_df[no_state_df['Shipping Address State'].isin(shipping_codes)]['Survey ResponseID'].nunique())

608 shipping codes
2 unique Response Ids with shipping codes


In [162]:
print('metrics for data before removing invalid states')
print_data_metrics(amzn_df)

print('\nmetrics after removing invalid states')
amzn_df = amzn_df[(amzn_df['state'].notnull()) | amzn_df['Shipping Address State'].isna()]
print_data_metrics(amzn_df)

metrics for data before removing invalid states
5254 response Ids
1942117 total rows of data

metrics after removing invalid states
5254 response Ids
1939873 total rows of data


In [163]:
# Are there responseIds where there are no US states in the data? Yes.

no_states_response_ids_n = amzn_df['Survey ResponseID'].nunique() - amzn_df[amzn_df['state'].notnull()]['Survey ResponseID'].nunique()
print('%s Response IDs without any valid states' % no_states_response_ids_n)
# Get those Response IDs
no_states_response_ids = list(set(amzn_df['Survey ResponseID'].unique()) - set(amzn_df[amzn_df['state'].notnull()]['Survey ResponseID'].unique()))

# What does their data look like?
# How much data?
# After looking at this -- they might have just removed all address information and caught shipping state too
# for i, rid in enumerate(no_states_response_ids):
#     rdf = amzn_df[amzn_df['Survey ResponseID']==rid]
#     print('%s - %s: %s rows of data' % (i, rid, len(rdf)))
print('Here is the distribution of how many rows of data they have')
no_states_response_ids_rows = [len(amzn_df[amzn_df['Survey ResponseID']==rid]) for rid in no_states_response_ids]
pd.Series(no_states_response_ids_rows).describe()

66 Response IDs without any valid states
Here is the distribution of how many rows of data they have


count      66.000000
mean      443.484848
std       660.169238
min         1.000000
25%        69.750000
50%       275.500000
75%       444.250000
max      3207.000000
dtype: float64

## Deduplicate Amazon data
Collect duplicate response ids
using initial rows of data from Jan 2018

Methodology:
- Note data are already clipped to starting after 1-1-2018
- Get 1st X=5 (Order Date, ASIN) for each ResponseId --> concatenate to string as orders_string
- Make map dedup_map: {orders_string: [ResponseId]} -- append to list if already there
- dedup_map values which are lists of > 1 value show duplicates


In [432]:
"""
Methodology notes

Bad attempt v1: first tried this with just sorting on Order Date -- bad result:
Number of unique order strings differs based on number of orders used
5061 when orders=3 unique order strings; 5062 when orders=5; 5063 when orders=6; 5077 when orders=10
Why the difference? Tested possible reasons: 
a. really different ppl with same initial orders
b. there were < n_orders when initially did survey then did again after buying more stuff
Real reason: 
When customers made multiple orders on the same day, the order of the items in the order history could vary
This resulted in different order strings.

Bad attempt v2: Just use first n product codes, sorted.
More collisions with more orders, duh

Bad attempt v2: Sort on both 'Order Date', 'ASIN/ISBN (Product Code)'
Problem: Some product codes start with 0; in some cases this zero was excluded in downloaded data
resulting in a different order string

Solution that works: Sort on both 'Order Date', 'ASIN/ISBN (Product Code)', use last X digits.
"""


# After much testing, this is what works, for reasons noted above.
def get_orders_string(df, n_orders=5):
    # example orders string: '2020-08-11:0IX1I3G6-2020-08-21:00WHXN3C-2020-08-21:074DJIUO-2020-08-21:07ZJV7TC-2020-08-21:0AXUCJP6'
    # Using last X digits of product code
    return '-'.join(
        df[['Order Date', 'ASIN/ISBN (Product Code)']]
        .sort_values(['Order Date', 'ASIN/ISBN (Product Code)']).head(n_orders).astype(str)
        .apply(lambda x: ':'.join([x['Order Date'], x['ASIN/ISBN (Product Code)'][-8:]]), axis=1).to_list()
    )

In [435]:
get_orders_string(rdf)

'2020-08-11:0IX1I3G6-2020-08-21:00WHXN3C-2020-08-21:074DJIUO-2020-08-21:07ZJV7TC-2020-08-21:0AXUCJP6'

In [246]:
# This was me. I tested multiple times with my own data.
# Found during dedup process.
amzn_df[amzn_df['Survey ResponseID']=='R_3pmMPapxGMCnM5y'].head()

,Order Date,Purchase Price Per Unit,Quantity,Shipping Address State,Title,ASIN/ISBN (Product Code),Category,Survey ResponseID,unit price,total price,yyyy-mm,state
0,2018-01-21,$17.52,1.0,NY,"Scott 1000 Sheets Per Roll Toilet Paper, 27 Ro...",B01NBYY28W,TOILET_PAPER,R_3pmMPapxGMCnM5y,17.52,17.52,2018-01,NY
1,2018-01-21,$28.99,1.0,NY,"VIVA Choose-A-Sheet* Paper Towels, White, Big ...",B01LFFGW5K,PAPER_TOWEL,R_3pmMPapxGMCnM5y,28.99,28.99,2018-01,NY
2,2018-01-28,$25.38,1.0,NY,"Godel, Escher, Bach: An Eternal Golden Braid",0465026850,ABIS_BOOK,R_3pmMPapxGMCnM5y,25.38,25.38,2018-01,NY
3,2018-03-04,$11.99,1.0,NY,"Utopia Towels Soft Cotton Machine Washable, Ex...",B01DLD4YY0,TOWEL,R_3pmMPapxGMCnM5y,11.99,11.99,2018-03,NY
4,2018-03-04,$86.66,1.0,NY,NaN,B06XC29HPY,NaN,R_3pmMPapxGMCnM5y,86.66,86.66,2018-03,NY


In [436]:
n_orders=5
dedup_map = dict()
dup_count = 0
response_ids = amzn_df['Survey ResponseID'].unique()
for i, rid in enumerate(response_ids):
    if i%500 == 0:
        print('i=%s: %s' % (i, pd.Timestamp.now()))
    rdf = amzn_df[amzn_df['Survey ResponseID']==rid]
    orders_string = get_orders_string(rdf, n_orders=n_orders)
    if orders_string in dedup_map:
        dedup_map[orders_string] += [(rid, len(rdf))]
        dup_count += 1
        continue
    dedup_map[orders_string] = [(rid, len(rdf))]

i=0: 2023-04-23 13:59:02.949943
i=500: 2023-04-23 13:59:27.020088
i=1000: 2023-04-23 13:59:51.392118
i=1500: 2023-04-23 14:00:16.004254
i=2000: 2023-04-23 14:00:40.555973
i=2500: 2023-04-23 14:01:04.982048
i=3000: 2023-04-23 14:01:29.454378
i=3500: 2023-04-23 14:01:53.856107
i=4000: 2023-04-23 14:02:18.274911
i=4500: 2023-04-23 14:02:42.731963
i=5000: 2023-04-23 14:03:07.223436


In [437]:
"""
dedup map looks like this
{'0140502017-0873516095-193938012X-B004S7EZR0-B00DGYHIMU': [('R_3I9Pu8iauEcOx9A', 127)], 
'B00KVL0SIM-B01LYOCVZF-B01NAJGGA2-B07585JXNZ-B07B41717Z': [('R_7UtikIBqeQHvnyN', 53)], 
...}
"""
print('%s unique order strings' % len(dedup_map))
print('vs %s unique response Ids' % len(response_ids))
print('= %s duplicate responses' % (dup_count))

5054 unique order strings
vs 5254 unique response Ids
= 200 duplicate responses


In [440]:
# Would there be fewer dups if we used more orders? Test dups again with 10
dedup_map10 = dict()
dup_count10 = 0
for i, (_orders_string5, rids) in enumerate(dedup_map.items()):
    if i%500 == 0:
        print('i=%s: %s: %s' % (i, pd.Timestamp.now(), len(rids)))
    if len(rids) < 2:
        continue
    for (rid, _n) in rids:
        rdf = amzn_df[amzn_df['Survey ResponseID']==rid]
        orders_string10 = get_orders_string(rdf, n_orders=10)
        if orders_string10 in dedup_map10:
            dedup_map10[orders_string10] += [(rid, len(rdf))]
            dup_count10 += 1
            continue
        dedup_map10[orders_string10] = [(rid, len(rdf))]

i=0: 2023-04-23 14:04:05.649727: 1
i=500: 2023-04-23 14:04:07.790638: 1
i=1000: 2023-04-23 14:04:09.649859: 1
i=1500: 2023-04-23 14:04:11.461137: 1
i=2000: 2023-04-23 14:04:13.223879: 1
i=2500: 2023-04-23 14:04:14.913610: 1
i=3000: 2023-04-23 14:04:17.849260: 1
i=3500: 2023-04-23 14:04:19.997409: 1
i=4000: 2023-04-23 14:04:21.609771: 1
i=4500: 2023-04-23 14:04:23.567915: 1
i=5000: 2023-04-23 14:04:24.641705: 1


In [444]:
print('vs %s duplicate responses' % dup_count10)

vs 199 duplicate responses


In [442]:
# find the difference
diff_dups = dict()
m = 0
for orders_string5, rids5 in dedup_map.items():
    # find the corresponding orders string (n=10)
    for orders_string10, rids10 in dedup_map10.items():
        if orders_string10.startswith(orders_string5):
            m += 1
            if len(rids5) != len(rids10):
                if orders_string5 not in diff_dups:
                    diff_dups[orders_string5] = dict()
                diff_dups[orders_string5][orders_string10] = rids10

Here we see only difference is due to one string being longer than the other (more orders).

So use dedup_map with n_orders=5

In [443]:
diff_dups

{'2018-03-31:07BLNK6C-2018-03-31:0XAF0AIE-2018-05-21:74F2K187-2018-05-21:74T9LMC2-2018-06-10:00YB70PS': {'2018-03-31:07BLNK6C-2018-03-31:0XAF0AIE-2018-05-21:74F2K187-2018-05-21:74T9LMC2-2018-06-10:00YB70PS-2018-06-17:00YB70PS': [('R_3HFzy82OvxJnm7L',
    6)],
  '2018-03-31:07BLNK6C-2018-03-31:0XAF0AIE-2018-05-21:74F2K187-2018-05-21:74T9LMC2-2018-06-10:00YB70PS-2018-06-17:00YB70PS-2018-09-02:1IWM70JQ-2018-11-05:7CP99SBT-2018-11-05:7DLSQFL8-2018-11-20:6XVZ6F2C': [('R_3EF584Zf9pAwkir',
    201)]}}

Collect the list of response ids to drop vs keep from the duplicates lists.

How to choose?
Keep the ones with the most data. Take the first if they're the same.
Where Most data: Most rows with no nan values.

In [461]:
drop_rids = []
for _i, (_orders_string, rids) in enumerate(dedup_map.items()):
    if len(rids) > 1:
        keep_rid = None
        most_rows = 0
        for (rid, _n) in rids:
            notna_rows = len(amzn_df[amzn_df['Survey ResponseID']==rid].dropna())
            if notna_rows >= most_rows: # some have nan state in all rows and will hence have notna_rows = 0 -- keep
                most_rows = notna_rows
                if keep_rid is not None:
                    drop_rids += [keep_rid]
                keep_rid = rid
            else:
                drop_rids += [rid]

print(len(drop_rids))
# drop_rids

200


In [466]:
drop_responseids = drop_rids

### Drop Amazon data from duplicate responses

In [469]:
print('Before dropping duplicate Amazon data:')
print('%s rows of Amazon data' % len(amzn_df))
print('%s unique response ids' % amzn_df['Survey ResponseID'].nunique())

Before dropping duplicate Amazon data:
1939873 rows of Amazon data
5254 unique response ids


In [471]:
amzn_df = amzn_df[~amzn_df['Survey ResponseID'].isin(drop_responseids)]
print('Before dropping duplicate Amazon data:')
print('%s rows of Amazon data' % len(amzn_df))
print('%s unique response ids' % amzn_df['Survey ResponseID'].nunique())

Before dropping duplicate Amazon data:
1860434 rows of Amazon data
5054 unique response ids


## Process Survey Data

Output: processed survey data (can be used on its own)
- Restrict to valid worker/Prolific IDs
- Deduplicate on worker/Prolific IDs
- Join the 2 survey datasets
- Remove data where 'test' is indicated
- Remove rows with bad ResponseIds (from step 1)
- Make public
    - separate comments
    - remove worker/Prolific IDs


In [669]:
survey_unprocessed_fpath = '../data/survey-data/unprocessed/'
v0_survey_unprocessed_fpath = survey_unprocessed_fpath + 'v0.csv'
cloudresearch_survey_unprocessed_fpath = survey_unprocessed_fpath + 'v-cloudresearch.csv'

### Deduplicate Cloudresearch survey data

In [721]:
cloudresearch_survey_df = pd.read_csv(cloudresearch_survey_unprocessed_fpath)
# first rows is fields/description of columns
print('%s rows of data before preprocessing' % len(cloudresearch_survey_df.drop([0])))
print('removing duplicate worker IDs -- keeping last')
cloudresearch_survey_df = cloudresearch_survey_df.drop_duplicates(
    subset='Q-workerId', keep='last')
print('%s rows of data' % len(cloudresearch_survey_df.drop([0])))
# cloudresearch_survey_df.drop([0]).head()

793 rows of data before preprocessing
removing duplicate worker IDs -- keeping last
792 rows of data


### Deduplicate v0/Prolific survey data

- Survey was initially used in mturk
- Did not collect mturk worker IDs, instead gave mturk workers a random code upon completion
- No reason to believe possible for duplicate submissions under a single mturk worker ID
- There may have been duplicate Prolific IDs in responses -- find and remove those

In [722]:
v0_survey_df = pd.read_csv(v0_survey_unprocessed_fpath)
# first rows is fields/description of columns
print('%s rows of data before preprocessing' % len(v0_survey_df.drop([0])))
# Find and remove any duplicate Prolific IDs
# Get rows with non-na Prolific IDs; find duplicates, keep last
# v0_survey_df.drop([0])

5733 rows of data before preprocessing


In [723]:
prolific_ids = v0_survey_df['Q-prolific'].dropna()
print('%s Prolific ID entries' % len(prolific_ids))
# prolific_ids
# Duplicate entries are here:
# prolific_ids[prolific_ids.duplicated(keep='last')].index
print('Dropping %s duplicates' % prolific_ids.duplicated(keep='last').sum())
v0_survey_df = v0_survey_df.drop(prolific_ids[prolific_ids.duplicated(keep='last')].index)
print('%s remaining rows of data' % len(v0_survey_df.drop([0])))

5704 Prolific ID entries
Dropping 15 duplicates
5718 remaining rows of data


### Merge survey data

#### First save the fields

In [724]:
fields_fpath = '../data/survey-data/fields.csv'

cloudresearch_fields = cloudresearch_survey_df.loc[0].rename('fields').to_frame()
v0_fields = v0_survey_df.loc[0].rename('fields').to_frame()
v0_fields.loc['Q-workerId'] = cloudresearch_fields.loc['Q-workerId'] #'What is your Worker ID?'
v0_fields.loc['connect'] = cloudresearch_fields.loc['connect'] # Embedded data
v0_fields.to_csv(fields_fpath)
print('saved fields to file %s' % fields_fpath)
v0_fields

saved fields to file ../data/survey-data/fields.csv


,fields
Duration (in seconds),Duration (in seconds)
RecordedDate,Recorded Date
ResponseId,Response ID
Q-prolific,What is your Prolific ID?
Q-prolific-mturk,Do you also complete surveys (or HITs) using A...
q-demos-age,What is your age group?
Q-demos-hispanic,"Are you of Spanish, Hispanic, or Latino origin?"
Q-demos-race,Choose one or more races that you consider you...
Q-demos-education,What is the highest level of education you hav...
Q-demos-income,What was your total household income before ta...


#### Merge

In [725]:
print('Expect %s rows after concatenating' % 
      (len(v0_survey_df.drop(0)) + len(cloudresearch_survey_df.drop(0))))

Expect 6510 rows after concatenating


In [726]:
# Merge
survey_df = pd.concat(
    [v0_survey_df.drop(0), cloudresearch_survey_df.drop(0)], 
    ignore_index=True # Otherwise have duplicate index values
)
print('%s rows from merged data' % len(survey_df))
# survey_df.head(3)

6510 rows from merged data


#### Remove test data

Remove any rows where test is in the worker/Prolific ID or comment

In [727]:
tests = survey_df[
    survey_df['Q-prolific'].apply(lambda pid: 'test' in str(pid)) \
    | survey_df['Q-workerId'].apply(lambda wid: 'test' in str(wid)) \
    | survey_df['Q-comments'].apply(lambda c: (len(str(c)) < 50) and ('test' in str(c).lower()))
]
tests

,Duration (in seconds),RecordedDate,ResponseId,Q-prolific,Q-prolific-mturk,q-demos-age,Q-demos-hispanic,Q-demos-race,Q-demos-education,Q-demos-income,...,Q-small-biz-use,Q-census-use,Q-research-society,Q-attn-check,Q-comments,ResponseID,showdata,incentive,Q-workerId,connect
23,192,10/4/2022 8:30:11,R_PBTmuy6D8TQbN97,NaN,NaN,25 - 34 years,No,White or Caucasian,Some high school or less,"$150,000 or more",...,I don't know,I don't know,No,"Yes,No,I don't know",test by alex for pilot-3 (did BATCH stick?),R_PBTmuy6D8TQbN97,false,altruism,NaN,NaN
24,188,10/4/2022 8:30:36,R_2whJTuUcscZje1i,NaN,NaN,25 - 34 years,No,White or Caucasian,"Graduate or professional degree (MA, MS, MBA, ...","$25,000 - $49,999",...,Yes,Yes,Yes,"Yes,No,I don't know",TEST RM,R_2whJTuUcscZje1i,true,control,NaN,NaN
25,799,10/11/2022 11:04:24,R_2ZD2bHi2XkFFA4D,NaN,NaN,35 - 44 years,No,White or Caucasian,Prefer not to say,Prefer not to say,...,I don't know,I don't know,I don't know,"Yes,No,I don't know",TEST ANDY,R_2ZD2bHi2XkFFA4D,true,bonus-20,NaN,NaN
5717,1212,4/16/2023 0:25:05,R_YWhHGZNwr5aUuRP,test,No,18 - 24 years,Yes,White or Caucasian,"Graduate or professional degree (MA, MS, MBA, ...","$25,000 - $49,999",...,No,No,Yes,"Yes,No,I don't know",NaN,R_YWhHGZNwr5aUuRP,false,bonus-05,NaN,NaN


In [728]:
survey_df = survey_df.drop(tests.index)
print('%s rows after dropping tests' % len(survey_df))

6506 rows after dropping tests


### Remove duplicate responses based on Amazon data

In [729]:
print('found %s response ids to drop from Amazon data' % len(drop_responseids))
print('%s to be dropped from survey response data' % survey_df['ResponseId'].isin(drop_responseids).sum())
survey_df = survey_df[~survey_df['ResponseId'].isin(drop_responseids)]
# drop_responseids

found 200 response ids to drop from Amazon data
181 to be dropped from survey response data


In [730]:
print('%s remaining survey data rows' % len(survey_df))

6325 remaining survey data rows


In [733]:
# Confirm the ResponseId and embedded data ResponseID are the same
assert(0 == len(survey_df[survey_df['ResponseId'] != survey_df['ResponseID']]))
# And drop extra column for ResponseID
survey_df = survey_df.drop('ResponseID', axis=1)

### Handle comments

As per the IRB protocol, separate the comments from the rest of the response data.

In [735]:
comments_fpath = '../data/survey-data/comments.csv'

comments = survey_df['Q-comments'].rename('comments').dropna().reset_index(drop=True).to_frame()
print('%s comments' % len(comments))
print('saving comments without response id and with different index to %s' % comments_fpath)
comments.to_csv(comments_fpath, index=False)
print('dropping comments from survey data')
survey_df = survey_df.drop('Q-comments', axis=1)
comments.head()

738 comments
saving comments without response id and with different index to ../data/survey-data/comments.csv
dropping comments from survey data


,comments
0,Great study!
1,none today
2,good survey
3,aaa
4,Thanks!


### Prepare the survey data to make publicly available

Drop worker/Prolific ID.
Drop extra metadata ResponseID.

In [736]:
processed_survey_fpath = '../data/survey-data/survey.csv'
survey_df = survey_df.drop(['Q-prolific', 'Q-workerId'], axis=1)
print('saving %s rows of survey data to %s' % (len(survey_df), processed_survey_fpath))
survey_df.to_csv(processed_survey_fpath, index=False)

saving 6325 rows of survey data to ../data/survey-data/survey.csv


In [737]:
survey_df = pd.read_csv(processed_survey_fpath)
survey_df.head()

,Duration (in seconds),RecordedDate,ResponseId,Q-prolific-mturk,q-demos-age,Q-demos-hispanic,Q-demos-race,Q-demos-education,Q-demos-income,Q-demos-gender,...,Q-data-value-any_1_TEXT,Q-sell-YOUR-data,Q-sell-consumer-data,Q-small-biz-use,Q-census-use,Q-research-society,Q-attn-check,showdata,incentive,connect
0,332,9/21/2022 10:00:17,R_1ou69fj4DQGsVcp,NaN,35 - 44 years,No,Black or African American,High school diploma or GED,"$25,000 - $49,999",Female,...,NaN,No,No,No,No,No,"Yes,No,I don't know",False,bonus-50,NaN
1,488,9/21/2022 10:02:24,R_24dboHVOzohx1kw,NaN,25 - 34 years,No,White or Caucasian,Bachelor's degree,"$25,000 - $49,999",Male,...,NaN,Yes if I get part of the profit,Yes,No,Yes,Yes,"Yes,No,I don't know",False,bonus-05,NaN
2,309,9/21/2022 10:10:47,R_2UbJL30HRjK1sdD,NaN,45 - 54 years,No,White or Caucasian,High school diploma or GED,"$100,000 - $149,999",Male,...,NaN,No,No,No,No,Yes,"Yes,No,I don't know",False,bonus-05,NaN
3,339,9/21/2022 10:13:03,R_UPXamGKtmf4RVIZ,NaN,25 - 34 years,No,White or Caucasian,High school diploma or GED,"$25,000 - $49,999",Male,...,NaN,No,No,No,No,Yes,"Yes,No,I don't know",False,bonus-50,NaN
4,674,9/21/2022 11:36:51,R_2dYk5auG9Fv5Qve,NaN,35 - 44 years,Yes,White or Caucasian,"Graduate or professional degree (MA, MS, MBA, ...","$50,000 - $74,999",Male,...,NaN,No,No,No,No,No,"Yes,No,I don't know",True,control,NaN


## Output cleaned Amazon data

Remove any rows without a corresponding Response id in survey data.
- Due to dropping duplicates and tests from survey data
- Due to incomplete responses
- Due to failed attention checks

In [739]:
survey_response_ids = survey_df['ResponseId']
print('%s survey response ids' % len(survey_response_ids))
assert(len(survey_response_ids) == survey_df['ResponseId'].nunique())
print('%s unique response ids in Amazon data' % amzn_df['Survey ResponseID'].nunique())
print('%s unique response ids in both amazon data and survey data' % (
    amzn_df[amzn_df['Survey ResponseID'].isin(survey_response_ids)]['Survey ResponseID'].nunique()))

# Drop Amazon data that does not have a correponding responseId in survey data
print('dropping Amazon data without corresponding responseId in survey data')
amzn_df = amzn_df[amzn_df['Survey ResponseID'].isin(survey_response_ids)]
print('%s total Amazon data rows' % len(amzn_df))

6325 survey response ids
5054 unique response ids in Amazon data
5027 unique response ids in both amazon data and survey data
dropping Amazon data without corresponding responseId in survey data
1850717 total Amazon data rows


In [740]:
cleaned_amz_data_fpath = '../data/amazon-data/amazon-data-cleaned.csv'
print('saving cleaned amazon data to %s' % cleaned_amz_data_fpath)
amzn_df.to_csv(cleaned_amz_data_fpath)

saving cleaned amazon data to ../data/amazon-data/amazon-data-cleaned.csv


Data links for restricted access data

(Permissioned to people with approvals)

- [Survey comments](https://drive.google.com/file/d/1E3o9-rYRvf0ZYRzMYXUhJ8PVeVztcen2/view?usp=share_link)
- [Amazon data](https://drive.google.com/file/d/12Od0Rl6zVPecrAOlH9JEIuyKeQixFYno/view?usp=share_link)
